In [ ]:
#!pip install selenium

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.chrome.options import Options

In [3]:
options = Options()
#options.add_argument("--headless")  # Run without UI
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")

In [27]:
driver = webdriver.Chrome(options = options)

In [29]:
choice = int(input("""Which language do you prefer? Select 1.Telugu  2.Hindi 3.English """))
if choice == 1:
    choice = 'Telugu'
elif choice == 2:
    choice = 'Hindi'
else:
    choice = 'English'

choice = choice.lower()
url = f'https://in.bookmyshow.com/explore/movies-hyderabad?languages={choice}'

Which language do you prefer? Select 1.Telugu  2.Hindi 3.English  3


In [31]:
driver.get(url)
driver.implicitly_wait(30)

In [33]:
data = bs(driver.page_source , 'html.parser')

In [35]:
anchors = data.find_all('a', class_="sc-133848s-11 sc-1ljcxl3-1 cxWSPX")
#anchors
print(f"Found {len(anchors)} movie links")

Found 15 movie links


In [39]:
result = []
base_url = "https://in.bookmyshow.com"
for mv in anchors:
    #print(mv)
    #print('\n\n')

    
    mv_link = base_url + mv.get('href') if mv.get('href').startswith('/') else mv.get('href')
    src = mv.find('img').get('src') 
    try:
        movie_name = mv.find('h3').text.strip()
    except:
        pass
    try:
        if movie_name.lower().replace(' ', '-').replace('(','').replace(')','') in mv_link:
            #print(movie_name.lower())
            #print(mv_link)

            item = {
        
                'name': movie_name,
                'image': src,
                'url':mv_link
            }
            print(item)
            result.append(item)
            #print("\n\n")
    except Exception as e:
        print(f"Skipped a movie due to missing data: {e}")
    

Skipped a movie due to missing data: name 'movie_name' is not defined
{'name': 'A Complete Unknown', 'image': 'https://assets-in.bmscdn.com/discovery-catalog/events/tr:w-400,h-600,bg-CCCCCC:w-400.0,h-660.0,cm-pad_resize,bg-000000,fo-top:l-image,i-discovery-catalog@@icons@@star-icon-202203010609.png,lx-24,ly-615,w-29,l-end:l-text,ie-OC44LzEwICAxODUgVm90ZXM%3D,fs-29,co-FFFFFF,ly-612,lx-70,pa-8_0_0_0,l-end/et00434215-vgxqrwwgty-portrait.jpg', 'url': 'https://in.bookmyshow.com/hyderabad/movies/a-complete-unknown/ET00434215'}
{'name': 'The Brutalist', 'image': 'https://assets-in.bmscdn.com/discovery-catalog/events/tr:w-400,h-600,bg-CCCCCC:w-400.0,h-660.0,cm-pad_resize,bg-000000,fo-top:l-image,i-discovery-catalog@@icons@@star-icon-202203010609.png,lx-24,ly-615,w-29,l-end:l-text,ie-OC4yLzEwICAxOTQgVm90ZXM%3D,fs-29,co-FFFFFF,ly-612,lx-70,pa-8_0_0_0,l-end/et00422382-hcrcycymhx-portrait.jpg', 'url': 'https://in.bookmyshow.com/hyderabad/movies/the-brutalist/ET00422382'}
{'name': 'Mickey 17', 'ima

In [ ]:
#sending an email

In [41]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
#mime for formatting the html text

In [43]:
sender_email = "likitha418@gmail.com"  
sender_password = "fcge ocsx dycx mkxl"  
receiver_email = [ "likitha418@gmail.com" ] 
subject = "BookMyShow Movies Running Today"

In [ ]:
html_body = """
<html>
<head>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        h2 { color: #333; text-align: center; }
        ul { list-style-type: none; padding: 0; }
        li { margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9; }
        .movie-title { font-weight: bold; font-size: 18px; color: #2a2a2a; }
        .movie-link { color: #0066cc; text-decoration: none; }
        .movie-link:hover { text-decoration: underline; }
        .movie-image { display: block; margin: 10px auto; max-width: 150px; }
        p { text-align: center; color: #666; }
    </style>
</head>
<body>
<h2>Telugu Movie Data from BookMyShow</h2>
<ul>
"""

In [ ]:
movies = result
for movie in movies:
    html_body += f"""
    <li>
        <div class="movie-title">{movie['name']}</div>
        <a href="{movie['url']}" class="movie-link">Book Now</a><br>
        <img src="{movie['image']}" alt="{movie['name']}" class="movie-image">
    </li>
    """

In [ ]:
html_body += """
</ul>
<p>Total movies: {}</p>
</body>
</html>
""".format(len(movies))

In [ ]:
for person in receiver_email:
  
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = person
    
    # Attach HTML content
    html_part = MIMEText(html_body, "html")
    msg.attach(html_part)
    
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()  # Enable TLS
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, person, msg.as_string())
        print(f"Email sent to {person} successfully!")
    except Exception as e:
        print(f"Failed to send email to {person}: {e}")

In [ ]:
driver.quit()